In [2]:
#!pip install -U openai-whisper

import whisper

In [4]:
model = whisper.load_model("large")

/Users/tmedeirosb/Desktop/DEV/LANGCHAIN/venv/lib/python3.10/site-packages/whisper/__init__.py:65: UserWarning: /Users/tmedeirosb/.cache/whisper/large-v3.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(
100%|█████████████████████████████████████| 2.88G/2.88G [00:52<00:00, 59.1MiB/s]


In [5]:
result = model.transcribe("/Users/tmedeirosb/Downloads/Abertura_Design_Instrucional_e_Producao_de_Material_Didatico.mp4") 
print(result['text'])

/Users/tmedeirosb/Desktop/DEV/LANGCHAIN/venv/lib/python3.10/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Você está no... Pessoal, bom dia a todos e a todas. Hoje é o nosso primeiro encontro síncrono. É muito importante que vocês prestem atenção que são duas disciplinas que estão ocorrendo ao mesmo tempo, certo? Processo de produção de material didático e design institucional. Os alunos confundem muito isso e acabam fazendo as atividades só de um. E esquecem a outra, tá bem? Então, a gente vai se apresentar. E a gente vai apresentar as duas páginas. O primeiro módulo, tirar as dúvidas, tá bem? Eu sou o Thiago Medeiros, sou professor aqui do Instituto Federal há quase 13 anos. Minha formação é... Eu sou engenheiro de computação, meu doutorado também nessa área, mas foco até em inteligência artificial, ciência de dados na área de educação. Mas também tenho formações na área pedagógica. Tenho a licenciatura em formação pedagógica e também tenho uma especialização... em design institucional. E já atuei como diretor de produção de material didático e de educação à distância aqui no IFRN por qu

In [6]:
with open ('whisper_text.txt', 'w') as file:   
    file.write(result['text'])

In [8]:
from langchain import LLMChain 
from langchain.llms import Ollama
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

In [9]:
# Initialize language model
llm = Ollama(model='llama3:latest', temperature=0)

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

text_splitter = RecursiveCharacterTextSplitter(    
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"])

texts = text_splitter.split_text(result['text'])
docs = [Document(page_content=t) for t in texts[:4]]

In [11]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

chain = load_summarize_chain(llm, chain_type="map_reduce")
output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)


/Users/tmedeirosb/Desktop/DEV/LANGCHAIN/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
/Users/tmedeirosb/Desktop/DEV/LANGCHAIN/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Here is a concise summary:  The speakers, Thiago Medeiros and Louise, introduce themselves as
experts in education and pedagogy. They welcome students to a synchronous meeting covering two
disciplines: producing educational materials and institutional design. The goal is to share
experiences and perspectives on using the platform, with an emphasis on autonomy and time
management.


In [14]:
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS in PT-BR:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

In [15]:
chain = load_summarize_chain(llm,
                             chain_type="stuff",
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary,
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Aqui está uma síntese concisa em pontos:

* O professor Thiago Medeiros apresenta-se e explica que o objetivo é compartilhar sua experiência em processo de produção de material didático e design institucional.
* A colega Louise também se apresenta, mostrando sua formação em pedagogia e doutorado em educação.
* As duas disciplinas estão ocorrendo ao mesmo tempo e é importante que os alunos prestem atenção para não confundir as atividades.
* O objetivo é compartilhar experiências e conhecimentos entre os professores e os estudantes.
* A dinâmica do curso será de apresentação das disciplinas, com oportunidade de uso do chat e abertura da câmera e áudio para tirar dúvidas.
* Os estudantes são convidados a se sentirem à vontade e contribuir com suas vivências e experiências.


In [16]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

Here is a refined summary:  The speakers, Thiago Medeiros and Louise, welcome students to their
first synchronous meeting, emphasizing the importance of paying attention to both disciplines being
covered simultaneously: process of producing educational material and institutional design. They
introduce themselves as an engineer with a background in computer science, education, and pedagogy,
and a professor with a background in pedagogy and education.  Thiago emphasizes the need for
students to be actively engaged in their learning process, encouraging them to use the chat feature
to ask questions or share their thoughts. He also invites students to open their camera and audio if
they have any doubts or want to contribute to the discussion.  Louise highlights the importance of
autonomy among students, emphasizing the need to keep track of deadlines and submit activities on
time. She also shares her experience using the platform and expresses excitement about starting the
course.  The spe